In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime 
import calendar
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [4]:
sv = pd.read_csv('US_Accidents_Dec20_updated.csv')
sv.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-2716600,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.10891,-83.09286,40.11206,-83.03187,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2716601,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.86542,-84.06280,39.86501,-84.04873,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-2716602,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10266,-84.52468,39.10209,-84.52396,0.055,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-2716603,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10148,-84.52341,39.09841,-84.52241,0.219,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
4,A-2716604,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.06213,-81.53784,41.06217,-81.53547,0.123,At Dart Ave/Exit 21 - Accident.,...,False,False,False,False,False,False,Night,Night,Day,Day


In [3]:
#To check if any null values exist in the dataset
#format(sv.isnull().values.any())
#To check if any null values exist in the dataset
#format(sv.isnull().values.any())

In [5]:
#detects the missing values in the dataset
sv.isnull().sum()

ID                             0
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                        0
End_Lng                        0
Distance(mi)                   0
Description                    0
Number                   1046095
Street                         0
Side                           0
City                          83
County                         0
State                          0
Zipcode                      935
Country                        0
Timezone                    2302
Airport_Code                4248
Weather_Timestamp          30264
Temperature(F)             43033
Wind_Chill(F)             449316
Humidity(%)                45509
Pressure(in)               36274
Visibility(mi)             44211
Wind_Direction             41858
Wind_Speed(mph)           128862
Precipitation(in)         510549
Weather_Condition          44007
Amenity   

In [6]:
#drop columns that are not required
sv_main=sv.drop(columns=['ID','End_Lat','End_Lng','Number',
                         'Wind_Chill(F)','Zipcode','Country','Timezone','Airport_Code',
                         'Wind_Direction','Weather_Timestamp','Temperature(F)','Humidity(%)',
                         'Pressure(in)','Precipitation(in)'])


In [7]:
#handling the missing values
sv_main['City']=sv_main['City'].fillna("Unknown")
sv_main['Sunrise_Sunset']=sv_main['Sunrise_Sunset'].fillna("Unknown")
sv_main['Civil_Twilight']=sv_main['Civil_Twilight'].fillna("Unknown")
sv_main['Nautical_Twilight']=sv_main['Nautical_Twilight'].fillna("Unknown")
sv_main['Astronomical_Twilight']=sv_main['Astronomical_Twilight'].fillna("Unknown")
sv_main['Weather_Condition']=sv_main['Weather_Condition'].fillna("Unknown")
sv_main['Visibility(mi)'].fillna(sv_main['Visibility(mi)'].median(), inplace=True)
sv_main['Wind_Speed(mph)'].fillna(sv_main['Wind_Speed(mph)'].median(), inplace=True)


sv_main=sv_main[sv_main.City != "Unknown"]
sv_main=sv_main[sv_main.Sunrise_Sunset != "Unknown"]
sv_main=sv_main[sv_main.Civil_Twilight != "Unknown"]
sv_main=sv_main[sv_main.Nautical_Twilight != "Unknown"]
sv_main=sv_main[sv_main.Astronomical_Twilight != "Unknown"]
sv_main=sv_main[sv_main.Weather_Condition != "Unknown"]

In [8]:
sv_main

,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,Side,City,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.10891,-83.09286,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,Outerbelt E,R,Dublin,...,False,False,False,False,False,False,Night,Night,Night,Night
1,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.86542,-84.06280,0.747,At OH-4/OH-235/Exit 41 - Accident.,I-70 E,R,Dayton,...,False,False,False,False,False,False,Night,Night,Night,Night
2,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10266,-84.52468,0.055,At I-71/US-50/Exit 1 - Accident.,I-75 S,R,Cincinnati,...,False,False,False,False,False,False,Night,Night,Night,Day
3,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10148,-84.52341,0.219,At I-71/US-50/Exit 1 - Accident.,US-50 E,R,Cincinnati,...,False,False,False,False,False,False,Night,Night,Night,Day
4,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.06213,-81.53784,0.123,At Dart Ave/Exit 21 - Accident.,I-77 N,R,Akron,...,False,False,False,False,False,False,Night,Night,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516059,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.00248,-117.37936,0.543,At Market St - Accident.,Pomona Fwy E,R,Riverside,...,False,False,False,False,False,False,Day,Day,Day,Day
1516060,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.76696,-117.14806,0.338,At Camino Del Rio/Mission Center Rd - Accident.,I-8 W,R,San Diego,...,False,False,False,False,False,False,Day,Day,Day,Day
1516061,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.77545,-117.84779,0.561,At Glassell St/Grand Ave - Accident. in the ri...,Garden Grove Fwy,R,Orange,...,False,False,False,False,False,False,Day,Day,Day,Day
1516062,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.99246,-118.40302,0.772,At CA-90/Marina Fwy/Jefferson Blvd - Accident.,San Diego Fwy S,R,Culver City,...,False,False,False,False,False,False,Day,Day,Day,Day


In [9]:
new_sv = sv[['ID','Severity','Start_Time','Side','City','County','State','Visibility(mi)','Weather_Condition']]
new_sv

,ID,Severity,Start_Time,Side,City,County,State,Visibility(mi),Weather_Condition
0,A-2716600,3,2016-02-08 00:37:08,R,Dublin,Franklin,OH,10.0,Light Rain
1,A-2716601,2,2016-02-08 05:56:20,R,Dayton,Montgomery,OH,10.0,Light Rain
2,A-2716602,2,2016-02-08 06:15:39,R,Cincinnati,Hamilton,OH,10.0,Overcast
3,A-2716603,2,2016-02-08 06:15:39,R,Cincinnati,Hamilton,OH,10.0,Overcast
4,A-2716604,2,2016-02-08 06:51:45,R,Akron,Summit,OH,10.0,Overcast
...,...,...,...,...,...,...,...,...,...
1516059,A-4239402,2,2019-08-23 18:03:25,R,Riverside,Riverside,CA,10.0,Fair
1516060,A-4239403,2,2019-08-23 19:11:30,R,San Diego,San Diego,CA,10.0,Fair
1516061,A-4239404,2,2019-08-23 19:00:21,R,Orange,Orange,CA,10.0,Partly Cloudy
1516062,A-4239405,2,2019-08-23 19:00:21,R,Culver City,Los Angeles,CA,10.0,Fair


In [10]:
new_sv['time'] = pd.to_datetime(new_sv.Start_Time, format='%Y-%m-%d %H:%M:%S')
new_sv

d:\python jupiter\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,Severity,Start_Time,Side,City,County,State,Visibility(mi),Weather_Condition,time
0,A-2716600,3,2016-02-08 00:37:08,R,Dublin,Franklin,OH,10.0,Light Rain,2016-02-08 00:37:08
1,A-2716601,2,2016-02-08 05:56:20,R,Dayton,Montgomery,OH,10.0,Light Rain,2016-02-08 05:56:20
2,A-2716602,2,2016-02-08 06:15:39,R,Cincinnati,Hamilton,OH,10.0,Overcast,2016-02-08 06:15:39
3,A-2716603,2,2016-02-08 06:15:39,R,Cincinnati,Hamilton,OH,10.0,Overcast,2016-02-08 06:15:39
4,A-2716604,2,2016-02-08 06:51:45,R,Akron,Summit,OH,10.0,Overcast,2016-02-08 06:51:45
...,...,...,...,...,...,...,...,...,...,...
1516059,A-4239402,2,2019-08-23 18:03:25,R,Riverside,Riverside,CA,10.0,Fair,2019-08-23 18:03:25
1516060,A-4239403,2,2019-08-23 19:11:30,R,San Diego,San Diego,CA,10.0,Fair,2019-08-23 19:11:30
1516061,A-4239404,2,2019-08-23 19:00:21,R,Orange,Orange,CA,10.0,Partly Cloudy,2019-08-23 19:00:21
1516062,A-4239405,2,2019-08-23 19:00:21,R,Culver City,Los Angeles,CA,10.0,Fair,2019-08-23 19:00:21


In [11]:
sv_main['Start_Time'] = pd.to_datetime(sv_main['Start_Time'], infer_datetime_format=True)
sv_main['Day_of_Week'] = sv_main['Start_Time'].dt.day_name()

sv_main[[ 'Start_Time', 'Day_of_Week']].head()

,Start_Time,Day_of_Week
0,2016-02-08 00:37:08,Monday
1,2016-02-08 05:56:20,Monday
2,2016-02-08 06:15:39,Monday
3,2016-02-08 06:15:39,Monday
4,2016-02-08 06:51:45,Monday


In [12]:
cor = sv_main.corr()

#Correlation with output variable
cor_target = abs(cor["Severity"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.02]
relevant_features

Severity           1.000000
Start_Lat          0.079884
Start_Lng          0.178700
Distance(mi)       0.152425
Wind_Speed(mph)    0.057534
Crossing           0.054141
Junction           0.021939
Traffic_Signal     0.043221
Name: Severity, dtype: float64

In [13]:
imp_features =["Severity","Start_Lat","Start_Lng","Distance(mi)","Wind_Speed(mph)","Amenity","Crossing","Junction","Railway","Station","Stop","Traffic_Signal"]

In [14]:
df_copy1 = sv_main.filter(imp_features, axis=1)


In [15]:
from sklearn.model_selection import train_test_split
y_data = df_copy1['Severity'].values
X_data = df_copy1.drop('Severity', axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.33, random_state=42)
X_train.shape

(986226, 11)

In [16]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)
rfe = RFE(lr, n_features_to_select=1)
rfe.fit(X_train,y_train)
 
print ("Features sorted by their rank:")
print (sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), df_copy1.columns)))

Features sorted by their rank:
[(1, 'Amenity'), (2, 'Wind_Speed(mph)'), (3, 'Junction'), (4, 'Start_Lng'), (5, 'Crossing'), (6, 'Station'), (7, 'Stop'), (8, 'Severity'), (9, 'Start_Lat'), (10, 'Railway'), (11, 'Distance(mi)')]


In [17]:

y_pred = lr.predict(X_test)
r2_score(y_test,y_pred)

0.06285035630429936

In [18]:

df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

,Actual,Predicted
0,2,2.285267
1,2,2.371873
2,2,2.100710
3,2,2.240272
4,2,2.398404
...,...,...
485749,2,2.028135
485750,3,2.521468
485751,2,2.398097
485752,4,2.332262


In [19]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.40227799922602747
Mean Squared Error: 0.3474955089389753
Root Mean Squared Error: 0.5894874968470284


In [20]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed, shuffle =True)
model = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(model,X_train,y_train, cv=kfold, scoring=scoring)

d:\python jupiter\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
d:\python jupiter\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extr

In [21]:
print("Accuracy for Logistic Regression :", results.mean())

Accuracy for Logistic Regression : 0.799239728409677


In [22]:
bool_features = df_copy1.select_dtypes(include=['bool'])
print('Boolean features: {}'.format(bool_features.shape))
one_hot = pd.get_dummies(bool_features)
one_hot.reset_index(drop=True, inplace=True)
one_hot.head(5)

Boolean features: (1471980, 7)


,Amenity,Crossing,Junction,Railway,Station,Stop,Traffic_Signal
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False
3,False,False,True,False,False,False,False
4,False,False,False,False,False,False,False


In [23]:
numerical_features =  df_copy1.select_dtypes(exclude=['bool'])
sc = StandardScaler()
y = df_copy1[['Severity']].values
y= sc.fit_transform(y)
numerical_features = numerical_features.drop(['Severity'], axis=1)
numerical_features = pd.DataFrame(sc.fit_transform(numerical_features), columns = numerical_features.columns)

In [24]:
X_data = np.concatenate((one_hot,numerical_features),axis=1)
df_concat = pd.concat([one_hot,numerical_features],axis=1)

In [25]:
X_train1,X_test1,y_train1,y_test1 = train_test_split(X_data, y, test_size=0.2, random_state=42)
y_t=y_train1.ravel()
y_te=y_test1.ravel()

In [27]:
pip install windrose

Note: you may need to restart the kernel to use updated packages.


'd:\python' is not recognized as an internal or external command,
operable program or batch file.


In [28]:
import lightgbm as lgb

In [29]:
lgb_train = lgb.Dataset(X_train1, y_t)
lgb_test = lgb.Dataset(X_test1, y_te)

In [30]:
params = {
          'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': {'rmse'},
            'num_leaves': 13,
            'max_depth': -1,
          'learning_rate': 0.8,
          'feature_fraction': 0.041,
          'bagging_fraction': 0.331,
          'bagging_freq' : 5
          }

In [31]:
gbm_regress = lgb.train(params,
              lgb_train,
              num_boost_round=4000,
              valid_sets=(lgb_train, lgb_test),
            early_stopping_rounds=20,
            verbose_eval = 20)


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029997 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 856
[LightGBM] [Info] Number of data points in the train set: 1177584, number of used features: 11
[LightGBM] [Info] Start training from score -0.000172
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Li

In [32]:
y_pred=gbm_regress.predict(X_test1)

In [33]:
from sklearn.metrics import mean_squared_error

score = np.sqrt(mean_squared_error(y_test1, y_pred))
print("The Test RMSE score for our model:", score)

The Test RMSE score for our model: 0.9282735690334531
